In [2]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import itertools
from scipy.optimize import curve_fit

random.seed(4)
# Reducing the size of the network

df_edges = pd.read_csv("com-Amazon.csv", delimiter = " ")
OG = nx.from_pandas_edgelist(df_edges, source="From", target="To", create_using= nx.DiGraph) #the original co-purchasing network
edges = random.sample(list(OG.edges()), int(OG.number_of_edges() * 0.15)) # get 15% of the edges in the graph
G=nx.DiGraph()
G.add_edges_from(edges)
largest_cc = max(nx.weakly_connected_components(G), key=len) #find the largest weakly component
G1=G.subgraph(largest_cc) # the final graph with the largest weakly connected component
G2= G1.to_undirected()
G2_node_list= [] # add node that have the degree > 2
for node,degree in G1.degree():
    if degree > 2:
        G2_node_list.append(node)
G2= G1.subgraph(G2_node_list) #creating the graph base on the degree > 2
        
removed_edges = random.sample(list(G2.edges()), int(G2.number_of_edges() * 0.2)) #we will removed 20% of the edges
#print(removed_edges) # tuples of node i, node j
G_train = G2.copy()
G_train.remove_edges_from(removed_edges)
G_test = G2.copy()
#G_test.add_edges_from(removed_edges)

G_train=G_train.to_undirected()


#common neighbor





In [6]:
# jacard
jacard = nx.jaccard_coefficient(G_train)
jacard_pred = []
for u, v, p in jacard:
    jacard_pred.append([u, v, p])
jacard_pred = pd.DataFrame(jacard_pred, columns=['u', 'v', 'jacard'])
jacard_pred = jacard_pred.sort_values(by='jacard', ascending=False)
jacard_pred = jacard_pred.reset_index(drop=True)
#jacard_pred.loc[jacard_pred['u'].isin(G_test.nodes()) & jacard_pred['v'].isin(G_test.nodes()), 'label'] = 1
jacard_pred['pred'] = 0
jacard_pred.loc[jacard_pred['jacard'] > 0.15, 'pred'] = 1
pred= []
for i in range(len(jacard_pred)): 
    if jacard_pred['pred'][i] == 1: # number of predicion 
        comparing= (int(jacard_pred.iloc[i]['u']),int(jacard_pred.iloc[i]['v']),(jacard_pred.iloc[i]['jacard']))
        pred.append(comparing)
success=0
failure=0
true=[]
for u, v, j in pred:
    if ((u,v)) in G_test.edges():
      true.append(j)
      success+=1
    else:
      failure+=1
print(success)

20


In [10]:
# adamic adar
adamic = nx.adamic_adar_index(G_train)
adamic_pred= []
for u, v, p in adamic:
    adamic_pred.append([u,v,p])
adamic_pred = pd.DataFrame(adamic_pred, columns=['u', 'v', 'adamic'])
adamic_pred = adamic_pred.sort_values(by='adamic', ascending=False)
adamic_pred = adamic_pred.reset_index(drop=True)
adamic_pred['pred'] = 0
adamic_pred.loc[adamic_pred['adamic'] > 0.15, 'pred'] = 1


pred=[]
for i in range(len(adamic_pred)): 
    if adamic_pred['pred'][i] == 1: # number of predicion 
        comparing= (int(adamic_pred.iloc[i]['u']),int(adamic_pred.iloc[i]['v']),(adamic_pred.iloc[i]['adamic']))
        pred.append(comparing)
success=0
failure=0
true=[]
for u, v, j in pred:
    if ((u,v)) in G_test.edges():
      true.append(j)
      success+=1
    else:
      failure+=1
print(success)

22
